# Post block assignment 3: Multi-level particle swarm optimisation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from celluloid import Camera
pd.options.mode.chained_assignment = None

%matplotlib widget

# Optimization problem functions

In [2]:
# functions to define optimization problem
def objective_inner(x,y):
    f_x_y = -1*(y+47)*np.sin((abs(y + (x/2) +47))**0.5) - x*np.sin((abs(x-(y+47)))**0.5)
    return f_x_y

def bound_constraint(x,y): # returns 1 for constraint passed and 0 for constrain failed.
    # is x coord within bounds?
    if x >= -512 and x <= 512:
        pos_x = True
    else:
        pos_x = False

    # is y coord within bounds?
    if y >= -512 and y <= 512:
        pos_y = True
    else:
        pos_y = False

    return pos_y*pos_x

bound_constraint(512,404.2319)  

1

In [3]:
# randomly initialise swarm for inner PSO
def initialisation_inner(swarm_size,upper_bound, lower_bound):
    # define dataframe to track swarm
    df_swarm = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

    # for each particle random initialise position, velocity and particle best known location
    for i in range(swarm_size):
        # two position coords
        x_i_x = round(random.uniform(lower_bound,upper_bound),5)
        x_i_y = round(random.uniform(lower_bound,upper_bound),5)

        # two velocity components,  init velocity to square root of position
        v_i_x = 0#round(abs(random.uniform(lower_bound,upper_bound))**0.25 ,5)
        v_i_y = 0#round(abs(random.uniform(lower_bound,upper_bound))**0.25 ,5)

        # particle objective function, is also at first this particles best known solution
        p_i = objective_inner(x_i_x,x_i_y)

        # save to dataframe
        df_swarm = df_swarm.append({'Particle':i,'x_i':[x_i_x,x_i_y],'v_i':[v_i_x,v_i_y],'P_i':[x_i_x,x_i_y],'f_P_i':p_i,'f_x_i':p_i},ignore_index=True)

    # find best initial particle
    global_best_particle = df_swarm['f_P_i'].argmin()
    df_global_best = df_swarm.iloc[global_best_particle]

    return df_swarm,df_global_best

df_swarm,df_global_best = initialisation_inner(10, 512, -512)
display(df_swarm)
display(df_global_best)

,Particle,x_i,v_i,P_i,f_P_i,f_x_i
0,0,"[57.9589, 391.55781]","[0, 0]","[57.9589, 391.55781]",-193.499157,-193.499157
1,1,"[-368.23317, 199.06917]","[0, 0]","[-368.23317, 199.06917]",-371.475542,-371.475542
2,2,"[182.44758, 460.92686]","[0, 0]","[182.44758, 460.92686]",441.440812,441.440812
3,3,"[-226.23346, 163.59115]","[0, 0]","[-226.23346, 163.59115]",291.914389,291.914389
4,4,"[81.72939, 119.88935]","[0, 0]","[81.72939, 119.88935]",-176.511648,-176.511648
5,5,"[-424.66475, -447.72842]","[0, 0]","[-424.66475, -447.72842]",-563.706158,-563.706158
6,6,"[-38.59854, 389.0033]","[0, 0]","[-38.59854, 389.0033]",-428.105295,-428.105295
7,7,"[-5.45127, 360.09826]","[0, 0]","[-5.45127, 360.09826]",-382.108461,-382.108461
8,8,"[60.89724, -319.62736]","[0, 0]","[60.89724, -319.62736]",73.351155,73.351155
9,9,"[-132.74473, -331.44627]","[0, 0]","[-132.74473, -331.44627]",-66.679944,-66.679944


Particle                           5
x_i         [-424.66475, -447.72842]
v_i                           [0, 0]
P_i         [-424.66475, -447.72842]
f_P_i                       -563.706
f_x_i                       -563.706
Name: 5, dtype: object

In [4]:
# PSO algorithm

def PSO_inner(omega,c_1,c_2,swarm_size, verbose=1):

    # step 1 and 2: intialise swarm
    df_swarm,df_global_best = initialisation_inner(swarm_size, 512, -512)

    # step 4: iterate first iteration
    iteration = 0
    swarm_iterations = []
    iteration_best = [] # keep track of best objective function value
    stop_criteria = False

    # step 5: loop until stopping condition met
    while iteration <= 100 and stop_criteria == False:
        if verbose == 1:
            print(f'Iteration: {iteration}')
        
        # define new dataframe from each iteration
        df_swarm_new = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

        for particle in range(swarm_size):

            # update particle but ensure that it is inside bound constrained decision space
            constraint_passed = False
            count = 1
            while constraint_passed == False:

                # update rules
                rho_1 = random.uniform(0,1)#/count
                rho_2 = random.uniform(0,1)#/count

                # get previous iteration values
                v_prev_x = df_swarm['v_i'].iloc[particle][0]
                v_prev_y = df_swarm['v_i'].iloc[particle][1]

                x_prev_x = df_swarm['x_i'].iloc[particle][0]
                x_prev_y = df_swarm['x_i'].iloc[particle][1]  

                P_i_x = df_swarm['P_i'].iloc[particle][0]
                P_i_y = df_swarm['P_i'].iloc[particle][1]

                P_g_x = df_global_best['P_i'][0]
                P_g_y = df_global_best['P_i'][1]

                # step 5: execute velocity update 
                v_new_x = omega*v_prev_x + rho_1*c_1*(P_i_x-x_prev_x) + rho_2*c_2*(P_g_x-x_prev_x)
                v_new_y = omega*v_prev_y + rho_1*c_1*(P_i_y-x_prev_y) + rho_2*c_2*(P_g_y-x_prev_y)

                # if we keep getting stuck on a boundary move to best ever
                if count > 10:
                    v_new_x =  rho_2*c_2*(P_g_x-x_prev_x)
                    v_new_y =  rho_2*c_2*(P_g_y-x_prev_y)    

                # step 6: update position
                x_new_x = x_prev_x + v_new_x
                x_new_y = x_prev_y + v_new_y

                # check bound constraint on new position
                constraint_passed = bound_constraint(x_new_x,x_new_y)
                # print(f'Constraint passed: {constraint_passed}')

                if count > 50:
                    print(f'!!! Infinite loop on inner PSO, cant find valid position updates!!! on particle {particle}')
                    display(df_swarm)
                    return 0,0,0

                # limit to num iterations
                count += 1

            # compute objective function values for new positions
            f_x_new = objective_inner(x_new_x,x_new_y)

            # step 7: check if we have a new best location for this particle
            f_x_prev = df_swarm['f_x_i'].iloc[particle]

            if f_x_new < f_x_prev: # if true best new location for this particle
                P_i_x = x_new_x
                P_i_y = x_new_y
                f_P_i = f_x_new

                # save new swarm particle to new dataframe
                df_swarm_new = df_swarm_new.append({'Particle':particle,'x_i':[x_new_x,x_new_y],'v_i':[v_new_x,v_new_y],'P_i':[x_new_x,x_new_y],'f_P_i':f_x_new,'f_x_i':f_x_new},ignore_index=True)
            else:
                P_i_x = x_prev_x
                P_i_y = x_prev_y
                f_P_i = f_x_prev

                # save new swarm particle to new dataframe
                df_swarm_new = df_swarm_new.append({'Particle':particle,'x_i':[x_new_x,x_new_y],'v_i':[v_new_x,v_new_y],'P_i':[x_prev_x,x_prev_y],'f_P_i':f_x_prev,'f_x_i':f_x_new},ignore_index=True)

            # step 8: check if we have a new global best
            f_P_x = df_global_best['f_P_i']

            if f_x_new < f_P_x:
                # update best known values
                df_global_best.loc['x_i'] = [x_new_x,x_new_y]
                df_global_best.loc['v_i'] = [v_new_x,v_new_y]
                df_global_best.loc['P_i'] = [x_new_x,x_new_y]
                df_global_best.loc['f_P_i'] = f_x_new
                df_global_best.loc['f_x_i'] = f_x_new

    
        # last step, new swarm becomes current swarm
        df_swarm = df_swarm_new
        swarm_iterations.append(df_swarm.copy())
        iteration_best.append(round(f_P_x,5)) # store best known

        # check stop criteria: 10 iterations with now improvement in global best
        last_10 = set(iteration_best[-10:])

        if len(last_10) == 1 and iteration > 10: # ie last 10 iterations unchanged
            stop_criteria = True

        iteration += 1

        # what is the swarm average velocity each iteration?
        avg_velocity = np.mean([np.mean(df_swarm['v_i'].iloc[i]) for i in range(swarm_size)])
        # print(f'Swarm average velocity: {avg_velocity}')


    return swarm_iterations,iteration_best,df_global_best['f_P_i']

swarm_iterations,iteration_best,overall_best = PSO_inner(1,0.25,0.25,50)
    



Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16


In [5]:
iteration_best

[-670.3405,
 -670.3405,
 -670.3405,
 -683.28844,
 -694.81347,
 -704.03242,
 -704.66641,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903,
 -704.78903]

In [31]:
# best known
overall_best

-894.5709812863005

# Visualization

In [28]:
# plot decision space 
x = np.arange(-512,512,1)
y = np.arange(-512,512,1)
xx,yy = np.meshgrid(x,y)

zz = objective_inner(xx,yy)

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.plot_surface(xx,yy, zz, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim([-512,512])
ax.set_ylim([-512,512])



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-512.0, 512.0)

In [56]:
# plot decision space 
swarm_size = 50

fig,ax = plt.subplots(figsize=(8,8))
x = np.arange(-512,512,1)
y = np.arange(-512,512,1)
xx,yy = np.meshgrid(x,y)

zz = objective_inner(xx,yy)

camera = Camera(fig)

for i in range(len(swarm_iterations)):

    #plot contour

    # ax = plt.axes(projection='3d')
    ax.contourf(xx,yy, zz,levels=10, cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    # ax.set_zlabel('z')
    ax.set_xlim([-512,512])
    ax.set_ylim([-512,512])

    # plot swarm
    df = swarm_iterations[i]
    for j in range(swarm_size):
        x = df['x_i'].iloc[j][0]
        y = df['x_i'].iloc[j][1]  
        ax.plot([x],[y],'r*',label=f'Iteration: {i}')
        # ax.legend(loc=0)

    camera.snap()

animation = camera.animate(interval=500)
animation.save('PSO_new.gif',writer='imagemagick',dpi=400)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

MovieWriter imagemagick unavailable; using Pillow instead.


# Multi level optimisation

In [89]:
# randomly initialise swarm for outer PSO

def initialisation_outer(swarm_size):
    # define dataframe to track swarm
    df_swarm = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

    # for each particle random initialise position, velocity and particle best known location
    for i in range(swarm_size):
        print(f'Initialising particle {i+1} out of {swarm_size}')
        # three position coords
        x_i_omega = round(random.uniform(0,2),5)
        x_i_c_1 = round(random.uniform(0,2),5)
        x_i_c_2 = round(random.uniform(0,2),5)

        # two velocity components,  init velocity to square root of position
        v_i_omega = 0#ound(abs(random.uniform(0,1))**0.25 ,5)
        v_i_c_1 = 0#round(abs(random.uniform(0,1))**0.25 ,5)
        v_i_c_2 = 0#round(abs(random.uniform(0,1))**0.25 ,5)

        # particle objective function, is also at first this particles best known solution
        swarm_iterations,iteration_best,global_best = PSO_inner(x_i_omega,x_i_c_1,x_i_c_2,swarm_size=50,verbose=0)
        p_i = global_best

        # save to dataframe
        df_swarm = df_swarm.append({'Particle':i,'x_i':[x_i_omega,x_i_c_1,x_i_c_2],'v_i':[v_i_omega,v_i_c_1,v_i_c_2],'P_i':[x_i_omega,x_i_c_1,x_i_c_2],'f_P_i':p_i,'f_x_i':p_i},ignore_index=True)

    # find best initial particle
    global_best_particle = df_swarm['f_P_i'].argmin()
    df_global_best = df_swarm.iloc[global_best_particle]

    return df_swarm,df_global_best

df_swarm,df_global_best = initialisation_outer(10)

Initialising particle 1 out of 10
Initialising particle 2 out of 10
Initialising particle 3 out of 10
Initialising particle 4 out of 10
Initialising particle 5 out of 10
Initialising particle 6 out of 10
Initialising particle 7 out of 10
Initialising particle 8 out of 10
Initialising particle 9 out of 10
Initialising particle 10 out of 10


In [93]:
# PSO algorithm

# optimization function for outer PSO is the result of the inner PSO
def objective_outer(omega,c_1,c_2,swarm_size):
    swarm_iterations,iteration_best = PSO_inner(omega,c_1,c_2,swarm_size)
    return iteration_best[-1]

def bound_constraint_outer(x,y,z): # returns 1 for constraint passed and 0 for constrain failed.
    # is x coord within bounds?
    if x >= 0 and x <= 2:
        pos_x = True
    else:
        pos_x = False

    # is x coord within bounds?
    if y >= 0 and y <= 2:
        pos_y = True
    else:
        pos_y = False

    # is x coord within bounds?
    if z >= 0 and z <= 2:
        pos_z = True
    else:
        pos_z = False

    return pos_y*pos_x*pos_z

def PSO_outer(omega,c_1,c_2,swarm_size):

    # step 1 and 2: intialise swarm
    print('Performing initialization:\n')
    df_swarm,df_global_best = initialisation_outer(swarm_size)

    # step 4: iterate first iteration
    iteration = 0
    swarm_iterations = []
    iteration_best = [] # keep track of best objective function value
    stop_criteria = False

    # step 5: loop until stopping condition met
    while iteration <= 50 and stop_criteria == False:
        print(f'Iteration: {iteration}')
        
        # define new dataframe from each iteration
        df_swarm_new = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

        for particle in range(swarm_size):
            print(f'\t Particle {particle+1} out of {swarm_size}')
            constraint_passed = False
            count = 1
            while constraint_passed == False:
                # update rules
                rho_1 = random.uniform(0,1)
                rho_2 = random.uniform(0,1)

                # get previous iteration values
                v_prev_omega = df_swarm['v_i'].iloc[particle][0]
                v_prev_c_1 = df_swarm['v_i'].iloc[particle][1]
                v_prev_c_2 = df_swarm['v_i'].iloc[particle][2]

                x_prev_omega = df_swarm['x_i'].iloc[particle][0]
                x_prev_c_1 = df_swarm['x_i'].iloc[particle][1]  
                x_prev_c_2 = df_swarm['x_i'].iloc[particle][2]  

                P_i_omega = df_swarm['P_i'].iloc[particle][0]
                P_i_c_1 = df_swarm['P_i'].iloc[particle][1]
                P_i_c_2 = df_swarm['P_i'].iloc[particle][2]

                P_g_omega = df_global_best['P_i'][0]
                P_g_c_1 = df_global_best['P_i'][1]
                P_g_c_2 = df_global_best['P_i'][2]

                # step 5: execute velocity update 
                v_new_omega = omega*v_prev_omega + rho_1*c_1*(P_i_omega-x_prev_omega) + rho_2*c_2*(P_g_omega-x_prev_omega)
                v_new_c_1 = omega*v_prev_c_1 + rho_1*c_1*(P_i_c_1-x_prev_c_1) + rho_2*c_2*(P_g_c_1-x_prev_c_1)
                v_new_c_2 = omega*v_prev_c_2 + rho_1*c_1*(P_i_c_2-x_prev_c_2) + rho_2*c_2*(P_g_c_2-x_prev_c_2)

                if count > 10:
                    v_new_omega = rho_2*c_2*(P_g_omega-x_prev_omega)
                    v_new_c_1 = rho_2*c_2*(P_g_c_1-x_prev_c_1)
                    v_new_c_2 = rho_2*c_2*(P_g_c_2-x_prev_c_2)

                # step 6: update position
                x_new_omega = x_prev_omega + v_new_omega
                x_new_c_1 = x_prev_c_1 + v_new_c_1
                x_new_c_2 = x_prev_c_2 + v_new_c_2

                # check bound constraints on decision variables
                constraint_passed = bound_constraint_outer(x_new_omega,x_new_c_1,x_new_c_2)
                
                if count > 50:
                    print(f'Breaking loop: outer PSO cant find valid parameters, broke on particle {particle} Constraint check: {constraint_passed}')
                    display(df_swarm)
                    return swarm_iterations,iteration_best

                count += 1

            # compute objective function values for new positions 
            # print(f'Omega: {round(x_new_omega,3)}, c_1: {round(x_new_c_1,3)}, c_2: {round(x_new_c_2,3)}')
            swarm_iterations_inner,iteration_best_inner,global_best_inner = PSO_inner(x_new_omega,x_new_c_1,x_new_c_2,swarm_size=50,verbose=0)
            f_x_new = global_best_inner

            if swarm_iterations_inner == 0 and iteriteration_best_inneration_best == 0 and global_best_inner == 0:
                print('Stoping algo; hyper-parameters causing infinite loop')
                return 0,0

            # step 7: check if we have a new best location for this particle
            f_x_prev = df_swarm['f_x_i'].iloc[particle]

            if f_x_new < f_x_prev: # if true best new location for this particle
                P_i_omega = x_new_omega
                P_i_c_1 = x_new_c_1
                P_i_c_2 = x_new_c_2
                f_P_i = f_x_new

                # save new swarm particle to new dataframe
                df_swarm_new = df_swarm_new.append({'Particle':particle,'x_i':[x_new_omega,x_new_c_1,x_new_c_2],'v_i':[v_new_omega,v_new_c_1,v_new_c_2],'P_i':[x_new_omega,x_new_c_1,x_new_c_2],'f_P_i':f_x_new,'f_x_i':f_x_new},ignore_index=True)
            else:
                P_i_omega = x_prev_omega
                P_i_c_1 = x_prev_c_1
                P_i_c_2 = x_prev_c_2
                f_P_i = f_x_prev

                # save new swarm particle to new dataframe
                df_swarm_new = df_swarm_new.append({'Particle':particle,'x_i':[x_new_omega,x_new_c_1,x_new_c_2],'v_i':[v_new_omega,v_new_c_1,v_new_c_2],'P_i':[x_prev_omega,x_prev_c_1,x_prev_c_2],'f_P_i':f_x_prev,'f_x_i':f_x_new},ignore_index=True)

            # step 8: check if we have a new global best
            f_P_x = df_global_best['f_P_i']

            if f_x_new < f_P_x:
                # update best known values
                df_global_best['x_i'] = [x_new_omega,x_new_c_1,x_new_c_2]
                df_global_best['v_i'] = [v_new_omega,v_new_c_1,v_new_c_2]
                df_global_best['P_i'] = [x_new_omega,x_new_c_1,x_new_c_2]
                df_global_best['f_P_i'] = f_x_new
                df_global_best['f_x_i'] = f_x_new

        # last step, new swarm becomes current swarm
        df_swarm = df_swarm_new
        swarm_iterations.append(df_swarm.copy())
        iteration_best.append(round(f_P_x,5)) # store best known

        # check stop criteria: 10 iterations with now improvement in global best
        last_10 = set(iteration_best[-10:])

        if len(last_10) == 1 and iteration > 10: # ie last 10 iterations unchanged
            stop_criteria = True

        iteration += 1

        print(f'Iteration best: {iteration_best[-1]}')

    return swarm_iterations,iteration_best,df_global_best['x_i']

swarm_iterations, iteration_best,solution = PSO_outer(1,1,1,10)
    

Performing initialization:

Initialising particle 1 out of 10
Initialising particle 2 out of 10
Initialising particle 3 out of 10
Initialising particle 4 out of 10
Initialising particle 5 out of 10
Initialising particle 6 out of 10
Initialising particle 7 out of 10
Initialising particle 8 out of 10
Initialising particle 9 out of 10
Initialising particle 10 out of 10
Iteration: 0
	 Particle 1 out of 10
	 Particle 2 out of 10
	 Particle 3 out of 10
	 Particle 4 out of 10
	 Particle 5 out of 10
	 Particle 6 out of 10
	 Particle 7 out of 10
	 Particle 8 out of 10
	 Particle 9 out of 10
	 Particle 10 out of 10
Iteration best: -955.25515
Iteration: 1
	 Particle 1 out of 10
	 Particle 2 out of 10
	 Particle 3 out of 10
	 Particle 4 out of 10
	 Particle 5 out of 10
	 Particle 6 out of 10
	 Particle 7 out of 10
	 Particle 8 out of 10
	 Particle 9 out of 10
	 Particle 10 out of 10
Iteration best: -955.25515
Iteration: 2
	 Particle 1 out of 10
	 Particle 2 out of 10
	 Particle 3 out of 10
	 Parti

To do: 
- try implement velocity clamping to slow velocity down, I think this is causing the search to escape the bounds
- inertia < 0 should cause velocity to slow over time
- maybe after a particle gets stuck try:
    - only let global best influence next move
    - keep it at this position forever

Ways to deal with bound constraints:
- bounce back in negative direction, might be issues with next iteration going in same direction
- if leave he dies
- only go toward global best  

# refactor PSO code

In [54]:
# constraints for high- and low- level PSOs
def constraint_low_level(x):
    dimensions = len(x)
    dimension_check = [] # store booleans for pass or fail constraint
    for var in x:
        if var > -512 and var < 512:
            dimension_check.append(True)
        else:
            dimension_check.append(False)

    result = dimension_check[0]
    for i in range(dimensions-1):
        result = result*dimension_check[i+1]
    return result

def constraint_high_level(x):
    dimensions = len(x)
    dimension_check = [] # store booleans for pass or fail constraint
    for var in x:
        if var > 0 and var < 2:
            dimension_check.append(True)
        else:
            dimension_check.append(False)

    result = dimension_check[0]
    for i in range(dimensions-1):
        result = result*dimension_check[i+1]
    return result
    
# objective for low-level PSO
def objective_inner(vector):
    x = vector[0]
    y = vector[1]
    f_x_y = -1*(y+47)*np.sin((abs(y + (x/2) +47))**0.5) - x*np.sin((abs(x-(y+47)))**0.5)
    return f_x_y

# objective for high-level PSO: the converged objective function value for best particle from low-level PSO
def objective_outer(vector):
    df_swarm,df_global_best = initialisation_inner_grid(50, 512, -512)
    swarm_iterations,iteration_best,overall_best = PSO(omega=vector[0],c_1=vector[1],c_2=vector[2],swarm_size=50, verbose=0, df_swam_initial=df_swarm, df_global_best_initial=df_global_best,num_dimensions=2, objective=objective_inner, constraint=constraint_low_level)
    return overall_best['f_x_i']

# initialise low level PSO
def initialisation_inner(swarm_size,upper_bound, lower_bound):
    # define dataframe to track swarm
    df_swarm = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

    # for each particle random initialise position, velocity and particle best known location
    for i in range(swarm_size):
        # two position coords
        x_i_x = round(random.uniform(lower_bound,upper_bound),5)
        x_i_y = round(random.uniform(lower_bound,upper_bound),5)

        # two velocity components,  init velocity to square root of position
        v_i_x = 0
        v_i_y = 0

        # particle objective function, is also at first this particles best known solution
        p_i = objective_inner([x_i_x,x_i_y])

        # save to dataframe
        df_swarm = df_swarm.append({'Particle':i,'x_i':[x_i_x,x_i_y],'v_i':[v_i_x,v_i_y],'P_i':[x_i_x,x_i_y],'f_P_i':p_i,'f_x_i':p_i},ignore_index=True)

    # find best initial particle
    global_best_particle = df_swarm['f_P_i'].argmin()
    df_global_best = df_swarm.iloc[global_best_particle]

    return df_swarm,df_global_best

def initialisation_inner_grid(swarm_size,upper_bound, lower_bound):
    # define dataframe to track swarm
    df_swarm = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

    x = np.linspace(-500,500,7)
    y = np.linspace(-500,500,7)

    xx,yy = np.meshgrid(x,y)

    x_parts = xx.flatten()
    y_parts = yy.flatten()

    x_parts = np.append(x_parts,0)#np.append(x_parts,round(random.uniform(-512,512),5))
    y_parts = np.append(y_parts,0)#np.append(y_parts,round(random.uniform(512,512),5))


    # for each particle random initialise position, velocity and particle best known location
    for i in range(swarm_size):
        # two position coords
        x_i_x = x_parts[i]
        x_i_y = y_parts[i]

        # two velocity components,  init velocity to square root of position
        v_i_x = 0
        v_i_y = 0

        # particle objective function, is also at first this particles best known solution
        p_i = objective_inner([x_i_x,x_i_y])

        # save to dataframe
        df_swarm = df_swarm.append({'Particle':i,'x_i':[x_i_x,x_i_y],'v_i':[v_i_x,v_i_y],'P_i':[x_i_x,x_i_y],'f_P_i':p_i,'f_x_i':p_i},ignore_index=True)

    # find best initial particle
    global_best_particle = df_swarm['f_P_i'].argmin()
    df_global_best = df_swarm.iloc[global_best_particle]

    return df_swarm,df_global_best

def initialisation_outer(swarm_size):
    # define dataframe to track swarm
    df_swarm = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

    # for each particle random initialise position, velocity and particle best known location
    for i in range(swarm_size):
        print(f'Initialising particle {i+1} out of {swarm_size}')
        # three position coords
        x_i_omega = round(random.uniform(0,2),5)
        x_i_c_1 = round(random.uniform(0,2),5)
        x_i_c_2 = round(random.uniform(0,2),5)

        # two velocity components
        v_i_omega = 0
        v_i_c_1 = 0
        v_i_c_2 = 0

        # particle objective function, is also at first this particles best known solution
        global_best = objective_outer([x_i_omega,x_i_c_1,x_i_c_2])
        p_i = global_best

        # save to dataframe
        df_swarm = df_swarm.append({'Particle':i,'x_i':[x_i_omega,x_i_c_1,x_i_c_2],'v_i':[v_i_omega,v_i_c_1,v_i_c_2],'P_i':[x_i_omega,x_i_c_1,x_i_c_2],'f_P_i':p_i,'f_x_i':p_i},ignore_index=True)

    # find best initial particle
    global_best_particle = df_swarm['f_P_i'].argmin()
    df_global_best = df_swarm.iloc[global_best_particle]

    return df_swarm,df_global_best



In [55]:
# PSO algorithm
def PSO(omega,c_1,c_2,swarm_size, df_swam_initial, df_global_best_initial,num_dimensions, objective, constraint, verbose=1):

    # step 1 and 2: intialise swarm
    df_swarm = df_swam_initial
    df_global_best = df_global_best_initial

    # initialise lists and variabls for loop
    iteration = 0           # initialise interations
    swarm_iterations = [df_swarm]   # keep track of each iterations swarm
    iteration_best = []     # keep track of best objective function value
    stop_criteria = False

    # Swarm iterator and updates: keep looping until max iterations reached or stopping condition met
    while iteration <= 100 and stop_criteria == False:
        # print iteration number
        if verbose == 1:
            print(f'Iteration: {iteration}')
        
        # define new dataframe from each iteration
        df_swarm_new = pd.DataFrame(columns=['Particle','x_i','v_i','P_i','f_P_i','f_x_i'])

        # Step 4: Update rules for each particle
        for particle in range(swarm_size):

            # update particle but ensure that it is inside bound constrained decision space, so continously try update particle until we have an allow position
            constraint_passed = False
            count = 1
            while constraint_passed == False:
                new_position_vector = [] # store temp new decision variables
                new_velocity_vector = []
                prev_position_vector = [] 

                # Update each dimension of decision variables
                for dimension in range(num_dimensions):

                    # update rules
                    rho_1 = random.uniform(0,1)
                    rho_2 = random.uniform(0,1)

                    # get previous iteration values
                    v_prev = df_swarm['v_i'].iloc[particle][dimension]

                    x_prev = df_swarm['x_i'].iloc[particle][dimension]

                    P_i = df_swarm['P_i'].iloc[particle][dimension]

                    P_g = df_global_best['P_i'][dimension]

                    # step 5: update velocity
                    v_new = omega*v_prev + rho_1*c_1*(P_i-x_prev) + rho_2*c_2*(P_g-x_prev)

                    # if we keep getting stuck on a boundary move toward global best only
                    if count > 10:
                        v_new =  rho_2*c_2*(P_g-x_prev)

                    # step 6: update position
                    x_new = x_prev + v_new

                    # store vectors for testing update rules

                    new_position_vector.append(x_new)
                    new_velocity_vector.append(v_new)
                    prev_position_vector.append(x_prev)

                # check bound constraint on new position for particle
                constraint_passed = constraint(new_position_vector) # if this is True then this particle is successfully updated and move on to next particle

                # if we try update this particle 50 times unsuccesfully then it is stuck in an infinite loop
                if count > 50:
                    print(f'!!! Cant find valid position updates!!! on particle {particle}')
                    display(df_swarm)
                    return 0,0,0

                # count number of times we try to update a particle position to a valid new local
                count += 1

            # compute objective function values for new positions
            f_x_new = objective(new_position_vector)

            # step 7: check if we have a new best location for this particle
            f_x_prev = df_swarm['f_x_i'].iloc[particle]

            if f_x_new < f_x_prev: # if true best new location for this particle
                # save new swarm particle to new dataframe
                df_swarm_new = df_swarm_new.append({'Particle':particle,'x_i':new_position_vector,'v_i':new_velocity_vector,'P_i':new_position_vector,'f_P_i':f_x_new,'f_x_i':f_x_new},ignore_index=True)
            else:
                # save new swarm particle to new dataframe
                df_swarm_new = df_swarm_new.append({'Particle':particle,'x_i':new_position_vector,'v_i':new_velocity_vector,'P_i':prev_position_vector,'f_P_i':f_x_prev,'f_x_i':f_x_new},ignore_index=True)

            # step 8: check if we have a new global best
            f_P_x = df_global_best['f_P_i']

            if f_x_new < f_P_x:
                # update best known values
                df_global_best.loc['x_i'] = new_position_vector
                df_global_best.loc['v_i'] = new_velocity_vector
                df_global_best.loc['P_i'] = new_position_vector
                df_global_best.loc['f_P_i'] = f_x_new
                df_global_best.loc['f_x_i'] = f_x_new

    
        # step 9: evaluate stoppping criteria 

        # new swarm becomes current swarm
        df_swarm = df_swarm_new
        swarm_iterations.append(df_swarm.copy())
        iteration_best.append(round(f_P_x,5)) # store best known particle for convergence
        # iteration_best.append(round(df_swarm['f_P_i'].mean(),2))

        # check stop criteria: 10 iterations with no improvement in global best
        last_10 = set(iteration_best[-10:])

        if len(last_10) == 1 and iteration > 10: # ie last 10 iterations unchanged
            stop_criteria = True

        # increment iteration counter
        iteration += 1

        if verbose == 1:
            print(f'Swarm best objective function value: {iteration_best[-1]}')
            best = df_global_best['x_i']
            print(f'Best known hyper-paramters: {best}\n')

    return swarm_iterations,iteration_best,df_global_best

# initialise
df_swarm,df_global_best = initialisation_inner_grid(50, 512, -512)

# call PSO
swarm_iterations,iteration_best,overall_best = PSO(omega=1,c_1=0.25,c_2=0.5,swarm_size=50, verbose=1, df_swam_initial=df_swarm, df_global_best_initial=df_global_best,num_dimensions=2, objective=objective_inner, constraint=constraint_low_level)
    
overall_best


Iteration: 0
Swarm best objective function value: -842.60502
Best known hyper-paramters: [333.33333333333326, 500.0]

Iteration: 1
Swarm best objective function value: -919.15035
Best known hyper-paramters: [468.3150695769548, 418.6967356395011]

Iteration: 2
Swarm best objective function value: -919.15035
Best known hyper-paramters: [468.3150695769548, 418.6967356395011]

Iteration: 3
Swarm best objective function value: -922.6144
Best known hyper-paramters: [481.94642311709555, 436.39615069863936]

Iteration: 4
Swarm best objective function value: -948.6273
Best known hyper-paramters: [509.31331003076446, 402.63052153338236]

Iteration: 5
Swarm best objective function value: -948.6273
Best known hyper-paramters: [509.31331003076446, 402.63052153338236]

Iteration: 6
Swarm best objective function value: -957.82422
Best known hyper-paramters: [511.491915344963, 403.99470223761733]

Iteration: 7
Swarm best objective function value: -958.32334
Best known hyper-paramters: [511.68769469111

Particle                                                 47
x_i                 [511.9999999999417, 404.23178389666236]
v_i         [2.015689126339198e-08, 0.00012700602769233284]
P_i                 [511.9999999999417, 404.23178389666236]
f_P_i                                              -959.641
f_x_i                                              -959.641
Name: 47, dtype: object

In [70]:
# run multi-level PSO

# initialise
df_swarm,df_global_best = initialisation_outer(10)

# call PSO
swarm_iterations,iteration_best,overall_best = PSO(omega=1,c_1=1,c_2=1,swarm_size=10, verbose=1, df_swam_initial=df_swarm, df_global_best_initial=df_global_best,num_dimensions=3, objective=objective_outer, constraint=constraint_high_level)
    

Initialising particle 1 out of 10
Initialising particle 2 out of 10
Initialising particle 3 out of 10
Initialising particle 4 out of 10
Initialising particle 5 out of 10
Initialising particle 6 out of 10
Initialising particle 7 out of 10
Initialising particle 8 out of 10
Initialising particle 9 out of 10
Initialising particle 10 out of 10
Iteration: 0
Swarm best objective function value: -959.64066
Best known hyper-paramters: [1.3686293764581619, 1.1670768537091574, 0.7609265521586618]

Iteration: 1
Swarm best objective function value: -959.64066
Best known hyper-paramters: [1.5668756663883787, 0.96808674779928, 0.7989238808504217]

Iteration: 2
Swarm best objective function value: -959.64066
Best known hyper-paramters: [1.5668756663883787, 0.96808674779928, 0.7989238808504217]

Iteration: 3
Swarm best objective function value: -959.64066
Best known hyper-paramters: [1.5668756663883787, 0.96808674779928, 0.7989238808504217]

Iteration: 4
Swarm best objective function value: -959.64066


In [66]:
swarm_iterations[-1]

,Particle,x_i,v_i,P_i,f_P_i,f_x_i
0,0,"[1.3616392658439316, 0.028176698115426488, 1.3...","[0.018202069053675145, 0.005179299830618199, -...","[1.3434371967902565, 0.02299739828480829, 1.33...",-959.640663,-888.793640
1,1,"[1.4717357961771427, 0.006916201611438322, 1.2...","[0.01850630365924281, 0.004071382668148566, 0....","[1.4717357961771427, 0.006916201611438322, 1.2...",-894.533942,-894.533942
2,2,"[1.337873865008823, 0.010474038570565378, 1.34...","[0.01864596052210571, -0.004806303029295941, -...","[1.3192279044867175, 0.01528034159986132, 1.35...",-959.640663,-888.948824
3,3,"[1.5254844829271796, 0.023432737594213462, 1.3...","[0.2970181617604883, 0.01740210349434401, -0.0...","[1.5254844829271796, 0.023432737594213462, 1.3...",-888.946886,-888.946886
4,4,"[1.3356815583413524, 0.01209647290695251, 1.54...","[-0.009560301427287768, 0.009387396283689071, ...","[1.3452418597686402, 0.0027090766232634386, 1....",-956.402984,-888.945680
5,5,"[1.5034102768463233, 0.004933363678255167, 1.3...","[-0.067539687975079, -0.004040191373219419, 0....","[1.5034102768463233, 0.004933363678255167, 1.3...",-959.640663,-959.640663
6,6,"[1.3652504519900186, 0.024520224198857403, 1.3...","[0.024707232952770164, 0.001814033873361026, -...","[1.3405432190372484, 0.022706190325496377, 1.3...",-956.348534,-888.946673
7,7,"[1.4337951494508707, 0.017993917327450044, 1.3...","[0.07595439721464096, -0.0008579007829683802, ...","[1.3578407522362297, 0.018851818110418424, 1.3...",-956.780171,-888.948861
8,8,"[1.338578573687417, 0.025866749430220454, 1.12...","[-0.0855375565745895, -0.0010361298941966727, ...","[1.4241161302620065, 0.026902879324417126, 0.8...",-956.077031,-888.946380
9,9,"[1.3679929557595507, 0.03650872459112592, 1.41...","[0.029891795612324648, 0.013352846387050029, -...","[1.3381011601472261, 0.02315587820407589, 1.50...",-959.640663,-888.948623


In [67]:
# initialise
df_swarm,df_global_best = initialisation_inner_grid(50, 512, -512)

# call PSO
swarm_iterations,iteration_best,overall_best = PSO(omega=overall_best['x_i'][0],c_1=overall_best['x_i'][1],c_2=overall_best['x_i'][2],swarm_size=50, verbose=1, df_swam_initial=df_swarm, df_global_best_initial=df_global_best,num_dimensions=2, objective=objective_inner, constraint=constraint_low_level)


Iteration: 0
Swarm best objective function value: -842.60502
Best known hyper-paramters: [333.33333333333326, 500.0]

Iteration: 1
Swarm best objective function value: -878.14076
Best known hyper-paramters: [340.3485764454308, 499.13695524673983]

Iteration: 2
Swarm best objective function value: -941.83447
Best known hyper-paramters: [485.7481464702478, 441.64052440926787]

Iteration: 3
Swarm best objective function value: -941.90324
Best known hyper-paramters: [511.8225404509586, 407.9138513685533]

Iteration: 4
Swarm best objective function value: -956.74679
Best known hyper-paramters: [511.5767927573737, 404.97113646491965]

Iteration: 5
Swarm best objective function value: -957.75051
Best known hyper-paramters: [511.93208894377756, 402.9576382325238]

Iteration: 6
Swarm best objective function value: -959.23861
Best known hyper-paramters: [511.89352558303864, 403.9494028091604]

Iteration: 7
Swarm best objective function value: -959.39043
Best known hyper-paramters: [511.926778265

In [73]:
swarm_iterations[-1]

,Particle,x_i,v_i,P_i,f_P_i,f_x_i
0,0,"[1.538559482638695, 1.1560810217923694, 0.7988...","[-0.12467557589276007, 0.04486202601819769, -0...","[1.538559482638695, 1.1560810217923694, 0.7988...",-954.948227,-954.948227
1,1,"[1.5665300434331988, 1.1062866399580675, 0.797...","[8.521696823333096e-05, 0.05189165263952382, -...","[1.5664448264649655, 1.0543949873185436, 0.800...",-954.983566,-888.948582
2,2,"[1.5100389009799868, 0.8970879066027159, 0.792...","[-0.04160426111595085, -0.09318483808542856, -...","[1.5100389009799868, 0.8970879066027159, 0.792...",-955.103969,-955.103969
3,3,"[1.5683970373312168, 0.9544882646184154, 0.799...","[-0.07272243436137656, -0.010899742823969447, ...","[1.5683970373312168, 0.9544882646184154, 0.799...",-959.640663,-959.640663
4,4,"[1.4962293863273424, 0.9924000240061325, 0.797...","[-0.09517035116985935, -0.011671678773934083, ...","[1.4962293863273424, 0.9924000240061325, 0.797...",-959.640663,-959.640663
5,5,"[1.9410569512434428, 0.7432612698400846, 0.544...","[0.16380022047476095, 0.03105986322409751, 0.1...","[1.777256730768682, 0.7122014066159871, 0.3789...",-956.724416,-950.067887
6,6,"[1.5719053334278696, 0.9663052296303019, 0.787...","[-0.0005445284326332728, -0.001747342786337236...","[1.572449861860503, 0.9680525724166391, 0.7764...",-954.490342,-888.940079
7,7,"[1.5668407972886724, 0.49777229185865723, 0.79...","[0.007363956699852117, 0.30663776354831496, 0....","[1.5594768405888204, 0.1911345283103423, 0.799...",-954.853918,-888.885380
8,8,"[1.6458808110528031, 0.91211734633314, 0.90265...","[-0.014879893426103182, 0.006763464733072532, ...","[1.6458808110528031, 0.91211734633314, 0.90265...",-959.640663,-959.640663
9,9,"[1.458476303405956, 0.7658531028795716, 0.8048...","[0.08135573261840631, -0.11043219655847197, 0....","[1.458476303405956, 0.7658531028795716, 0.8048...",-959.640663,-959.640663


In [71]:
# plot decision space as gif
swarm_size = 50

fig,ax = plt.subplots(figsize=(8,8))
x = np.arange(-515,515,1)
y = np.arange(-515,515,1)
xx,yy = np.meshgrid(x,y)

zz = objective_inner([xx,yy])

camera = Camera(fig)

for i in range(len(swarm_iterations)):

    #plot contour

    # ax = plt.axes(projection='3d')
    ax.contourf(xx,yy, zz,levels=10, cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    # ax.set_zlabel('z')
    ax.set_xlim([-515,515])
    ax.set_ylim([-515,515])

    # plot swarm
    df = swarm_iterations[i]
    for j in range(swarm_size):
        x = df['x_i'].iloc[j][0]
        y = df['x_i'].iloc[j][1]  
        ax.plot([x],[y],'r*',label=f'Iteration: {i}')
        # ax.legend(loc=0)

    camera.snap()

animation = camera.animate(interval=500)
animation.save('PSO_new.gif',writer='imagemagick',dpi=400)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: single positional indexer is out-of-bounds

In [72]:
len(swarm_iterations)

13

In [27]:
# plot decision space - grid
swarm_size = 50

fig,ax = plt.subplots(figsize=(8,8))
x = np.arange(-515,515,1)
y = np.arange(-515,515,1)
xx,yy = np.meshgrid(x,y)

zz = objective_inner([xx,yy])

for i in range(len(swarm_iterations)):

    #plot contour

    # ax = plt.axes(projection='3d')
    ax[i].contourf(xx,yy, zz,levels=10, cmap='viridis')
    ax[i].set_xlabel('x')
    ax.set_ylabel('y')
    # ax.set_zlabel('z')
    ax.set_xlim([-515,515])
    ax.set_ylim([-515,515])

    # plot swarm
    df = swarm_iterations[i]
    for j in range(swarm_size):
        x = df['x_i'].iloc[j][0]
        y = df['x_i'].iloc[j][1]  
        ax.plot([x],[y],'r*',label=f'Iteration: {i}')
        # ax.legend(loc=0)




array([-500.        , -333.33333333, -166.66666667,    0.        ,
        166.66666667,  333.33333333,  500.        , -500.        ,
       -333.33333333, -166.66666667,    0.        ,  166.66666667,
        333.33333333,  500.        , -500.        , -333.33333333,
       -166.66666667,    0.        ,  166.66666667,  333.33333333,
        500.        , -500.        , -333.33333333, -166.66666667,
          0.        ,  166.66666667,  333.33333333,  500.        ,
       -500.        , -333.33333333, -166.66666667,    0.        ,
        166.66666667,  333.33333333,  500.        , -500.        ,
       -333.33333333, -166.66666667,    0.        ,  166.66666667,
        333.33333333,  500.        , -500.        , -333.33333333,
       -166.66666667,    0.        ,  166.66666667,  333.33333333,
        500.        ])